In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the Boston Housing dataset
boston =fetch_california_housing()
X = boston.data  # Features
y = boston.target  # Labels (house prices)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Reshape to (n_samples, 1)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)

# Define the ANN model
class ANNRegression(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ANNRegression, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Model parameters
input_size = X.shape[1]  # Number of features (13 for Boston dataset)
hidden_size = 10         # Number of neurons in the hidden layer
output_size = 1          # Single output for regression (price)

# Initialize model, loss, and optimizer
model = ANNRegression(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test)

# Calculate regression metrics
y_pred_np = y_pred.numpy()
y_test_np = y_test.numpy()

mse = mean_squared_error(y_test_np, y_pred_np)
mae = mean_absolute_error(y_test_np, y_pred_np)
r2 = r2_score(y_test_np, y_pred_np)

print("\nRegression Metrics:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R2 Score: {r2:.2f}")


Epoch [100/1000], Loss: 0.6658
Epoch [200/1000], Loss: 0.4488
Epoch [300/1000], Loss: 0.4121
Epoch [400/1000], Loss: 0.3996
Epoch [500/1000], Loss: 0.3896
Epoch [600/1000], Loss: 0.3820
Epoch [700/1000], Loss: 0.3759
Epoch [800/1000], Loss: 0.3710
Epoch [900/1000], Loss: 0.3671
Epoch [1000/1000], Loss: 0.3630

Regression Metrics:
Mean Squared Error (MSE): 0.37
Mean Absolute Error (MAE): 0.43
R2 Score: 0.72


In [ ]:
### a deeper architecture below

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the California Housing dataset
california = fetch_california_housing()
X = california.data  # Features
y = california.target  # Labels (median house value)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Reshape to (n_samples, 1)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)

# Define the improved ANN model
class EnhancedANN(nn.Module):
    def __init__(self, input_size):
        super(EnhancedANN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # First hidden layer
        self.bn1 = nn.BatchNorm1d(64)        # Batch normalization
        self.act1 = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(64, 32)         # Second hidden layer
        self.bn2 = nn.BatchNorm1d(32)
        self.act2 = nn.LeakyReLU()
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(32, 16)         # Third hidden layer
        self.act3 = nn.LeakyReLU()

        self.fc4 = nn.Linear(16, 1)          # Output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.act3(x)

        x = self.fc4(x)
        return x

# Model parameters
input_size = X_train.shape[1]  # Number of features
model = EnhancedANN(input_size)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)  # Reduce LR every 100 epochs

# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test)

# Calculate regression metrics
y_pred_np = y_pred.numpy()
y_test_np = y_test.numpy()

mse = mean_squared_error(y_test_np, y_pred_np)
mae = mean_absolute_error(y_test_np, y_pred_np)
r2 = r2_score(y_test_np, y_pred_np)

print("\nRegression Metrics:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R2 Score: {r2:.2f}")


Epoch [100/1000], Loss: 0.4737
Epoch [200/1000], Loss: 0.4074
Epoch [300/1000], Loss: 0.3778
Epoch [400/1000], Loss: 0.3691
Epoch [500/1000], Loss: 0.3609
Epoch [600/1000], Loss: 0.3522
Epoch [700/1000], Loss: 0.3595
Epoch [800/1000], Loss: 0.3514
Epoch [900/1000], Loss: 0.3387
Epoch [1000/1000], Loss: 0.3385

Regression Metrics:
Mean Squared Error (MSE): 0.31
Mean Absolute Error (MAE): 0.38
R2 Score: 0.76
